# UKG to CMiC Integration Middleware
This python project pulls time entry data from UKG Time Management (UTM) along with associated employee information, transforms it into the necessary format for upload to CMiC pytimesheets, and posts the entries. Check log files for errors.

In [1]:

import utm_load
import cmic
import employee_push
import CMiC_Project_Import
from datetime import datetime

startdate = datetime(2025,4,6)
enddate = datetime(2025,4,12)
utm_load.load_ukg(startdate, enddate)

FileNotFoundError: [Errno 2] No such file or directory: 'DataFiles/DEPARTMENT MAP.csv'

In [ ]:
CMiC_Project_Import.load_cmic_projects()

Requesting offset 0...
Retrieved 50 records — hasMore: True
Requesting offset 50...
Retrieved 50 records — hasMore: True
Requesting offset 100...
Retrieved 50 records — hasMore: True
Requesting offset 150...
Retrieved 50 records — hasMore: True
Requesting offset 200...
Retrieved 50 records — hasMore: True
Requesting offset 250...
Retrieved 50 records — hasMore: True
Requesting offset 300...
Retrieved 50 records — hasMore: True
Requesting offset 350...
Retrieved 50 records — hasMore: True
Requesting offset 400...
Retrieved 50 records — hasMore: True
Requesting offset 450...
Retrieved 50 records — hasMore: True
Requesting offset 500...
Retrieved 50 records — hasMore: True
Requesting offset 550...
Retrieved 50 records — hasMore: True
Requesting offset 600...
Retrieved 50 records — hasMore: True
Requesting offset 650...
Retrieved 50 records — hasMore: True
Requesting offset 700...
Retrieved 50 records — hasMore: True
Requesting offset 750...
Retrieved 50 records — hasMore: True
Requesting 

In [ ]:
employee_push.main()

FileNotFoundError: [Errno 2] No such file or directory: 'DataFiles/DEPARTMENT MAP.csv'

In [ ]:
cmic.post_timesheets_to_CMiC()